# Разные способы получить ключевые слова

Так получилось, что я делала похожее задание раньше, так что я просто сдам вам эту тетрадку. Тут неправильно реализован индекс Жаккара и некоторые вещи я бы сейчас реализовала лучше, но, к сожалению, времени или сил править что-либо в нет. 

### Подготовка тестовых данных

Получаем тексты

In [1]:
!wget https://github.com/mannefedov/ru_kw_eval_datasets/raw/master/data/ng_0.jsonlines.zip
!wget https://github.com/mannefedov/ru_kw_eval_datasets/raw/master/data/ng_1.jsonlines.zip

--2021-06-30 13:24:39--  https://github.com/mannefedov/ru_kw_eval_datasets/raw/master/data/ng_0.jsonlines.zip
Распознаётся github.com (github.com)… 140.82.121.3
Подключение к github.com (github.com)|140.82.121.3|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://raw.githubusercontent.com/mannefedov/ru_kw_eval_datasets/master/data/ng_0.jsonlines.zip [переход]
--2021-06-30 13:24:39--  https://raw.githubusercontent.com/mannefedov/ru_kw_eval_datasets/master/data/ng_0.jsonlines.zip
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 2987145 (2,8M) [application/zip]
Сохранение в: «ng_0.jsonlines.zip»

ng_0.jsonlines.zip  100%[===================>]   2,85M  1,23MB/s    за 2,3s    

2021-06-30 13:24:42 (1,23 MB/

In [2]:
!unzip ng_0.jsonlines.zip 
!unzip ng_1.jsonlines.zip

Archive:  ng_0.jsonlines.zip
  inflating: ng_0.jsonlines          
Archive:  ng_1.jsonlines.zip
  inflating: ng_1.jsonlines          


In [9]:
PATH_TO_DATA = './'

In [12]:
import os
import pandas as pd

In [13]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA) if file.endswith('jsonlines')]

data = pd.concat([pd.read_json(file, lines=True) for file in files][:5], axis=0, ignore_index=True)

data.shape

(1987, 5)

In [14]:
data.head(3)

,keywords,title,url,content,summary
0,"[школа, образовательные стандарты, литература,...","Ольга Васильева обещала ""НГ"" не перегружать шк...",https://amp.ng.ru/?p=http://www.ng.ru/educatio...,В среду состоялось отложенное заседание Совета...,"Глава Минобрнауки считает, что в нездоровом аж..."
1,"[красота, законы]",У красоты собственные закон и воля,https://amp.ng.ru/?p=http://www.ng.ru/style/20...,"Хорошо, когда красота в глазах смотрящего живе...",О живительной пользе укорота при выборе между ...
2,"[юзефович, гражданская война, пепеляев, якутия]",Апокалиптический бунт,https://amp.ng.ru/?p=http://www.ng.ru/zavisima...,Когда-то Леонид Юзефович написал книгу о монго...,Крепость из тел и призрак независимой Якутии


In [19]:
texts = list(data.content)

In [20]:
import re
from pymystem3 import Mystem
m = Mystem()
from stop_words import get_stop_words

In [21]:
def get_mystem_lemmatized_text(text):
    """
    берет текст в виде строки и возвращает его 
    лемматизированным MyStem в виде списка слов
    """
    stop_words = set(get_stop_words('ru'))
    stop_words.update(['чей', 'свой', 'ежели', 'нешто', 'из-за'])
    lemmatized_text = [word for word in m.lemmatize(text)
                       if re.search('[а-яёa-z]', word)
                       and word not in stop_words]
    return lemmatized_text

In [22]:
mystem_lemmatized_texts = [get_mystem_lemmatized_text(text) for text in texts]

Получаем лемматизированные с помощью pymorphy2 тексты без стоп-слов

In [23]:
import re
from string import punctuation 
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from nltk import word_tokenize

In [24]:
def get_pymorphy2_lemmatized_text(text):
    """
    берет текст в виде строки и возвращает его без стоп-слов и
    лемматизированным pymorphy2 в виде списка слов
    """
    stop_words = set(get_stop_words('ru'))
    stop_words.update(['чей', 'свой', 'ежели', 'нешто', 'из-за'])
    lemmatized_text = [morph.parse(word)[0].normal_form.strip(punctuation+'.— ') 
                       for word in word_tokenize(text)
                       if re.search('[а-яёa-z]', word, flags=re.I)
                       and morph.parse(word)[0].normal_form.strip(punctuation+'.— ') 
                       not in stop_words]
    return lemmatized_text

In [25]:
pymorphy2_lemmatized_texts = [get_pymorphy2_lemmatized_text(text) for text in texts]

### 7 способов извлечения ключевых слов

Все способы подразумевают применение на лемматизированном тексте без стоп-слов

#### 1. Ключевые слова по частотности

In [26]:
from collections import Counter

In [27]:
def get_keywords_most_frequent(text, num_keywords):
    """
    берет текст в виде списка слов и количество ключевых слов для извлечения,
    возвращает в качестве ключевых слов самые частотные слова текста
    """
    keywords = [word_freq[0] for word_freq in Counter(text).most_common(num_keywords)]
    return keywords

In [28]:
for text in pymorphy2_lemmatized_texts:
    print(get_keywords_most_frequent(text, 5))

['фгос', 'стандарт', 'источник', 'образовательный', 'новый']
['красота', 'глаз', 'отчаяние', 'уйти', 'порыв']
['пепеляев', 'юзефович', 'книга', 'якутия', 'восстание']
['гонка', 'команда', 'пилот', 'ferrari', 'mclaren']
['есенин', 'поэт', 'клюев', 'смерть', 'сергей']
['медицина', 'медицинский', 'кафедра', 'выпускник', 'работать']
['книга', 'русский', 'мозг', 'островной', 'говор']
['ирак', 'война', 'американец', 'партизанский', 'войско']
['нейросеть', 'ai', 'клетка', 'искусственный', 'интеллект']
['россия', 'вступление', 'кристалина', 'георгиев', 'вто']
['фильм', 'приз', 'картина', 'кино', 'например']
['каша', 'сидеть', 'столовая', 'начинать', 'рок']
['место', 'псков', 'город', 'князь', 'андрей']
['индонезия', 'страна', 'мусульманский', 'экстремизм', 'терроризм']
['игра', 'говорить', 'большой', 'друг', 'парень']
['выборы', 'путин', 'президент', 'признавать', 'сша']
['водный', 'вода', 'качество', 'загрязнение', 'объект']
['школа', 'образование', 'москва', 'школьник', 'олимпиада']
['фильм'

['ребёнок', 'площадка', 'взрослый', 'лужники', 'спорт']
['ренэ', 'герр', 'воспоминание', 'книга', 'эссе']
['оркестр', 'фестиваль', 'ростропович', 'симфонический', 'klangverwaltung']
['рост', 'большой', 'эволюция', 'вид', 'наука']
['высоцкий', 'рассказ', 'говорить', 'песня', 'канал']
['хаспеть', 'цру', 'сша', 'американский', 'пытка']
['партия', 'марковцев', 'отделение', 'яблоко', 'конференция']
['руководитель', 'должный', 'районный', 'организация', 'предприятие']
['берия', 'народ', 'сталин', 'немец', 'кавказ']
['путин', 'президент', 'эксперт', 'интервью', 'вопрос']
['рейтинг', 'росатом', 'среди', 'хороший', 'работодатель']
['росатом', 'атомный', 'аэс', 'новый', 'российский']
['конкурс', 'управленческий', 'кириенко', 'опыт', 'пройти']
['хедж-фонд', 'рынок', 'фонд', 'цена', 'инвестиционный']
['закон', 'прокурор', 'дело', 'личность', 'журналист']
['мечеть', 'ditib', 'турецкий', 'германия', 'немецкий']
['третьяк', 'стих', 'книга', 'маяковский', 'стихотворение']
['закон', 'украина', 'донбасс

In [29]:
for text in mystem_lemmatized_texts:
    print(get_keywords_most_frequent(text, 5))

['фгос', 'стандарт', 'источник', 'образовательный', 'новый']
['красота', 'глаз', 'отчаяние', 'уходить', 'порыв']
['пепеляев', 'юзефович', 'книга', 'якутия', 'белый']
['команда', 'гонка', 'пилот', 'Ferrari', 'McLaren']
['есенин', 'поэт', 'клюев', 'смерть', 'сергей']
['медицина', 'медицинский', 'кафедра', 'выпускник', 'работать']
['книга', 'русский', 'мозг', 'островной', 'говор']
['ирак', 'война', 'американец', 'партизанский', 'войско']
['нейросеть', 'клетка', 'искусственный', 'интеллект', 'самый']
['россия', 'вступление', 'кристалина', 'вто', 'георгиев']
['фильм', 'приз', 'картина', 'кино', 'например']
['каша', 'сидеть', 'начинать', 'столовая', 'рок']
['место', 'псков', 'город', 'князь', 'андрей']
['индонезия', 'страна', 'мусульманский', 'экстремизм', 'терроризм']
['игра', 'становиться', 'говорить', 'друг', 'парень']
['выборы', 'путин', 'признавать', 'президент', 'сша']
['водный', 'вода', 'качество', 'загрязнение', 'объект']
['школа', 'образование', 'москва', 'высокий', 'школьник']
['фи

['собчак', 'крым', 'нга', 'выборы', 'штаб']
['военный', 'сбор', 'мужчина', 'информация', 'призывать']
['партия', 'выборы', 'гудков', 'собчак', 'яблоко']
['клетка', 'вода', 'нейрон', 'матрица', 'титан']
['выборы', 'путин', 'власть', 'избиратель', 'электорат']
['российский', 'этнический', 'население', 'народ', 'нация']
['цена', 'нефть', 'долл', 'доллар', 'руб']
['индия', 'сша', 'пакистан', 'россия', 'вашингтон']
['вопрос', 'рассуждение', 'новый', 'научный', 'представление']
['право', 'конкурс', 'произвол', 'знать', 'понимать']
['афганистан', 'афганский', 'абдулла', 'провинция', 'ганя']
['бояться', 'умберто', 'эко', 'литр', 'любить']
['словарь', 'электронный', 'бумажный', 'автор', 'новый']
['тележка', 'клиент', 'кофе', 'работа', 'доминик']
['цифровой', 'данные', 'новый', 'компания', 'энергетика']
['религиозный', 'радикальный', 'фсин', 'мусульманин', 'экстремизм']
['цивилизация', 'создавать', 'существо', 'разумный', 'вид']
['республика', 'дагестан', 'этнический', 'религиозный', 'группа']
[

['страна', 'россия', 'экологический', 'земля', 'показатель']
['пакистан', 'рф', 'асиф', 'страна', 'проходить']
['оон', 'грузия', 'конфликт', 'российский', 'мооннг']
['научный', 'академик', 'наука', 'кругляков', 'статья']
['дед', 'пихенький', 'эликсир', 'жидкость', 'гарик']
['молодежный', 'молодой', 'молодежь', 'движение', 'событие']
['партия', 'яблоко', 'женщина', 'политика', 'выборы']
['аркадьич', 'крокодил', 'африканский', 'место', 'геннадьич']
['отель', 'россия', 'турист', 'турецкий', 'русскоязычный']
['гонка', 'пилот', 'трасса', 'Ferrari', 'феттель']
['система', 'транспортный', 'цифровой', 'железный', 'дорога']
['мост', 'строительство', 'понтонный', 'уголь', 'хакасия']
['потребитель', 'мощность', 'аэс', 'цена', 'рынок']
['полет', 'экипаж', 'самолет', 'сирия', 'вопрос']
['медицинский', 'пункт', 'помощь', 'минздрав', 'фельдшерский']
['военный', 'войско', 'вдв', 'проходить', 'конкурс']
['россия', 'скрипаль', 'возможность', 'мэй', 'отравление']
['пий', 'папа', 'церковь', 'белардо', 'со

#### 2. Ключевые слова и биграммы по частотности

In [30]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [31]:
def get_keywords_bigrams_most_frequent(text, num_keywords):
    """
    берет текст в виде списка слов и количество ключевых слов для извлечения,
    возвращает в качестве ключевых слов самые частотные биграммы текста, 
    согласованные грамматически, и самые частотные слова 
    в зависимости от их нормированной частотности
    """
    bigrams = [(text[i], text[i + 1]) for i, word in enumerate(text) if i != (len(text) - 1)]
    freq_uninflected_bigrams_numbers = dict([bigram_freq for bigram_freq in Counter(bigrams).most_common(num_keywords)])
    freq_words_numbers = dict([(word_freq[0], word_freq[1] / 2) for word_freq in Counter(text).most_common(num_keywords)])
    freq_items_numbers = {**freq_uninflected_bigrams_numbers , **freq_words_numbers}
    freq_items_numbers = sorted(freq_items_numbers.items(), key=lambda item: item[1], reverse=True)[:num_keywords]
    keywords = []
    for item in freq_items_numbers:
        if type(item[0]) == tuple: 
            new_bigram = None
            first_word = morph.parse(item[0][0])[0]
            second_word = morph.parse(item[0][1])[0]
            if 'ADJF' in first_word.tag or 'ADJS' in first_word.tag:
                if second_word.tag.gender:
                    new_bigram = (first_word.inflect({second_word.tag.gender}).word, item[0][1])
            elif 'tran' in first_word.tag:
                if 'NOUN' in second_word.tag or 'ADJF' in first_word.tag or 'ADJS' in first_word.tag:
                    new_bigram = (item[0][0], second_word.inflect({'accs'}).word)
            elif 'intr' in first_word.tag and 'NOUN' in second_word.tag:
                if 'NOUN' in second_word.tag or 'ADJF' in first_word.tag or 'ADJS' in first_word.tag:
                    new_bigram = (item[0][0], second_word.inflect({'ablt'}).word)
            elif 'NOUN' in first_word.tag and 'NOUN' in second_word.tag:
                if 'Surn' not in first_word.tag and 'Name' not in first_word.tag:
                    new_bigram = (item[0][0], second_word.inflect({'gent'}).word)
            if new_bigram:
                keywords.append(' '.join(new_bigram))
            else:
                keywords.append(' '.join(item[0]))
        else:
             keywords.append(item[0]) 
    return keywords

In [32]:
for text in pymorphy2_lemmatized_texts:
    print(get_keywords_bigrams_most_frequent(text, 5))

['ольга васильев', 'исторический источник', 'фгос', 'стандарт', 'источник']
['красота', 'глаз', 'отчаяние', 'уйти', 'порыв']
['пепеляев', 'юзефович', 'книга', 'якутия', 'леонид юзефович']
['гонка', 'команда', 'пилот', 'ferrari', 'mclaren']
['есенин', 'поэт', 'клюев', 'смерть есенина', 'смерть']
['медицина', 'медицинский', 'кафедра', 'выпускник', 'практическую медицина']
['островного говор', 'книга', 'русский', 'русская островной', 'мозг']
['ирак', 'война', 'американец', 'партизанский', 'войско']
['нейросеть', 'ai', 'искусственный интеллект', 'клетка', 'искусственный']
['кристалина георгиев', 'россия', 'вступление', 'кристалина', 'георгиев']
['фильм', 'приз', 'картина', 'кино', 'например']
['каша', 'сидеть', 'исконно русский', 'столовая', 'начинать']
['место', 'князь андрея', 'псков', 'город', 'событийный туризм']
['индонезия', 'страна', 'экстремизм терроризма', 'мусульманская община', 'мусульманский']
['игра', 'говорить', 'большой', 'друг', 'парень']
['выборы', 'признавать выборы', 'пу

['девочка деньги', 'русский', 'поезд', 'деньга', 'девочка']
['газпром рассчитывать', 'формула цены', 'цена поставки', 'центральноазиатский', 'газпром']
['pony express', 'интернет-магазин', 'рынок', 'товар', 'pony']
['фестиваль', 'бетховен', 'цимерман', 'кристиан цимерман', 'анне-софи муттереть']
['гонка', 'команда', 'себастьян феттель', 'последний круг', 'монец']
['ленин', 'публицист', 'писать', 'маркс энгельс', 'шекспир достоевский']
['девочка деньги', 'русский', 'поезд', 'деньга', 'девочка']
['культура', 'политическую культура', 'политический', 'тип', 'респондент']
['енисей', 'мусор', 'производство', 'мешок мусора', 'красноярский край']
['выставочный центр', 'парк', 'проходить', 'выставочный', 'парк горького']
['единая россия', 'должный', 'кинематограф', 'кино', 'россия']
['дерево', 'тыс', 'тыс дерева', 'миллион дерева', 'дерево тыс']
['похоронный', 'история', 'ритуальная услуга', 'похоронное дело', 'смерть']
['сфера', 'специалист', 'работник', 'производство', 'зарплата']
['владимир 

['армен саркисян', 'серж саргсян', 'президент', 'саркисян', 'армения']
['культура', 'американец', 'терроризм', 'социальный', 'америка']
['центральную азия', 'россия китая', 'нурсултан назарбаев', 'некоторый', 'страна центральный']
['бог', 'стихотворение', 'поэт', 'религиозная поэзия', 'культурное подполье']
['самолёт', 'президент владимира', 'владимир путин', 'путин получить', 'получить доклад']
['троцкий', 'ленин', 'чернявский', 'революция', 'георгий чернявский']
['бортовый комплекс', 'комплекс обороны', 'оборона л370', 'л370 витебск', 'jane s']
['интернет', 'политика', 'газета', 'информация', 'сми']
['сотрудник', 'боевик', 'район', 'получить', 'ингушетия']
['добыча', 'россия', 'нефтедобыча', 'лукойл', 'федун']
['сочи', 'мероприятие', 'трасса', 'россия', 'билет']
['моравский', 'гражданская война', 'валериан иванович', 'сибирское правительство', 'верховный правитель']
['нефть', 'арктика', 'ледовитый океан', 'добыча', 'российский']
['фильм', 'млн руб', 'кино', 'проект', 'млн']
['владими

['барражировать боеприпасом', 'система', 'масса кг', 'боевой часть', 'аппарат']
['протопоп аввакума', 'рабочий группы', 'межстарообрядческое сотрудничество', 'церковь', 'празднование 400-летия']
['искусственный интеллект', 'интеллект', 'искусственный', 'функция', 'интеллектуальный']
['выборы', 'избирательный', 'молодой', 'молодого избиратель', 'избирком']
['проект', 'корабль', 'программа', 'кораблестроительная программа', 'вмф']
['москва строить', 'строить атомный', 'атомный авианосец', 'авианосец шторма', 'шторм немой']
['стих', 'поэт', 'дойти домом', 'дом', 'дом сойти']
['режиссёр', 'театр', 'создание', 'март белый', 'белый дом']
['необходимо', 'политика', 'россия', 'внешняя политика', 'нво']
['анкара', 'сша', 'турецкую власть', 'турецкий', 'турция']
['ученик школы', 'курганская область', 'пневматический пистолет', 'школа', 'пострадать']
['поставка', 'страна', 'россия', 'экспорт', 'нефть']
['матч', 'высокий', 'шайба', 'команда', 'американец']
['владимир', 'статуя', 'площадь', 'москва

In [33]:
for text in mystem_lemmatized_texts:
    print(get_keywords_bigrams_most_frequent(text, 5))

['исторический источник', 'ольга васильев', 'фгос', 'стандарт', 'источник']
['красота', 'глаз', 'отчаяние', 'красота уходить', 'уходить']
['пепеляев', 'юзефович', 'книга', 'якутия', 'белый']
['команда', 'гонка', 'пилот', 'ким райкконен', 'Ferrari']
['есенин', 'поэт', 'клюев', 'смерть есенина', 'смерть']
['медицина', 'медицинский', 'кафедра', 'выпускник', 'практическую медицина']
['островного говор', 'книга', 'русский', 'русская островной', 'мозг']
['ирак', 'война', 'американец', 'партизанский', 'войско']
['нейросеть', 'искусственный интеллект', 'клетка', 'искусственный', 'интеллект']
['кристалина георгиев', 'россия', 'вступление', 'кристалина', 'вто']
['фильм', 'приз', 'картина', 'кино', 'например']
['каша', 'сидеть', 'исконно русский', 'начинать', 'столовая']
['место', 'князь андрея', 'псков', 'город', 'событийный туризм']
['индонезия', 'страна', 'экстремизм терроризма', 'мусульманская община', 'мусульманский']
['игра', 'становиться', 'говорить', 'друг', 'парень']
['выборы', 'признава

['михаил иванович', 'михаил', 'иванович', 'новогодний', 'новый']
['электронный лом', 'лом', 'электронный', 'переработка', 'вниихт']
['абэ', 'япония', 'японский', 'отношение', 'северная корея']
['язык', 'китайский', 'мягкая сила', 'искусственный интеллект', 'китайский язык']
['канцлер фрг', 'депутат', 'меркель', 'большинство депутата', 'депутат бундестага']
['тайник', 'булыжник', 'место', 'альфовец', 'разведчик глубокий']
['бесконечный апрель', 'пьеса', 'веня', 'спектакль', 'самый']
['партия', 'кпрф', 'выборы', 'новый', 'социализм']
['академия науки', 'институт', 'тема', 'научный', 'наука']
['тереза', 'монахиня', 'монастырь', 'новый', 'создавать новый']
['нга', 'россия', 'путин', 'запад', 'российский']
['пожилой', 'дом престарелый', 'помощь', 'социальный', 'дом']
['олимпийский объект', 'компания', 'объект', 'иностранный', 'получение контракта']
['группа газа', 'газ', 'группа', 'нижний новгород', 'спортивный']
['рособрнадзор поставлять', 'поставлять оценку', 'оценка региональный', 'регио

['ленин', 'ф д', 'публицист', 'писать', 'маркс энгельс']
['девочка денег', 'русский', 'поезд', 'деньги', 'девочка']
['культура', 'политическую культура', 'политический', 'активистская культура', 'тип']
['енисей', 'мусор', 'производство', 'мешок мусора', 'красноярский край']
['парк', 'выставочный центр', 'сокольники', 'проходить', 'выставочный']
['единая россия', 'кинематограф', 'кино', 'самый', 'должный']
['дерево', 'тыс', 'тыс дерева', 'миллион дерева', 'дерево тыс']
['похоронный', 'становиться', 'история', 'ритуальная услуга', 'похоронное дело']
['сфера', 'специалист', 'работник', 'производство', 'отрасль']
['владимир федорович', 'федорович', 'владимир', 'вольдемар теодорович', 'научный']
['й статья', 'статья', 'центр совы', 'уголовное дело', 'дело']
['уральский мужик', 'видно поле', 'тыс руб', 'куйвашев', 'уральский']
['театр', 'опера', 'оперный театр', 'набукко', 'оперный']
['россия', 'россия европы', 'отношение', 'европа', 'сторона']
['узбекистан', 'ташкент', 'отношение', 'вопрос'

['восточная гута', 'руководство вооруженный', 'вооружённый исламистский', 'исламистское формирование', 'формирование джейши']
['грузинский марш', 'являться собственностью', 'христианская конфессия', 'член грузинский', 'православный храм']
['вид', 'млн', 'экологический', 'чужеродный вид', 'земля']
['результат рассмотрения', 'рассмотрение обращения', 'обращение раскаиваться', 'раскаиваться участником', 'участник религиозный']
['администрирование', 'налоговый', 'фнс', 'министерство финансов', 'взнос']
['встреча', 'патриарх', 'папа', 'лукашенко', 'белорусский президент']
['корея', 'кндр', 'южная корея', 'северная корея', 'сша']
['израильский', 'израиль', 'террорист', 'тыс шекеля', 'семья']
['путин', 'выборы', 'писать', 'россия', 'президентский выборы']
['россия', 'власть', 'страна', 'политический', 'тереза мэй']
['старый тоомас', 'датский город', 'путеводитель урмаса', 'урмас роом', 'книжка']
['безопасность', 'ишингер', 'ситуация', 'россия', 'мюнхенская конференция']
['российский', 'самоле

['продукция', 'предприятие', 'производство', 'объем', 'гражданская продукция']
['компания', 'представлять', 'бла', 'беспилотная система', 'беспилотный']
['ес', 'страна', 'отношение россии', 'франко германский', 'ряд вопроса']
['верховный рад', 'член цик', 'цик', 'лидер', 'состав']
['герой', 'книга', 'автор', 'россия', 'главный герой']
['болгария', 'патриарх', 'патриарх кирилла', 'президент', 'млрд доллара']
['конкурс', 'театр', 'отс', 'опера', 'георг отс']
['иск', 'суд', 'дело юкоса', 'арбитражный', 'компания']
['борьба организованный', 'организованный преступность', 'преступность особый', 'особое дело', 'бывший депутат']
['слуцкий', 'депутат', 'стоить', 'информация', 'телеканал дождя']
['доход', 'доход населения', 'население', 'денежный доход', 'реальный']
['россия', 'япония', 'компания', 'проект', 'точка зрения']
['россия', 'российский', 'сила', 'сирия', 'военный']
['газопровод', 'газ', 'турецкий поток', 'сирия', 'турция']
['страна', 'асеан', 'россия', 'соглашение', 'страна асеан']
[

['третьяков', 'стих', 'книга', 'стихотворение', 'маяковский']
['закон', 'украина', 'донбасс', 'россия', 'украинский']
['узбекистан', 'киргизия', 'вопрос', 'отношение', 'ташкент']
['регион', 'доход населения', 'доход', 'рост', 'зарплата']
['книга', 'наталья', 'презентация книги', 'наталья полякова', 'прочитывать стихотворение']
['центральный экологический', 'экологическая зона', 'озеро байкала', 'экологический', 'зона']
['греция', 'полуостров пелопоннеса', 'пожарный', 'пожар', 'район']
['самолет', 'крушение', 'сообщать', 'никакого обломок', 'обломок вертолёта']
['пепеляев', 'юзефович', 'книга', 'якутия', 'белый']
['всемирный совет', 'совет международный', 'международный автомобильный', 'автомобильная федерация', 'федерация автоспорта']
['золотой маска', 'премия золотой', 'показ москвы', 'прямой трансляция', 'москва апреля']
['наводнение', 'вода', 'проливного дождь', 'юго-восточная азия', 'чистая вода']
['российская авиация', 'военный', 'удар', 'бомбардировщик су', 'официальный представи

['выборы', 'избирательный', 'молодой', 'избирком', 'избирательная комиссия']
['проект', 'корабль', 'программа', 'кораблестроительная программа', 'фрегат проекта']
['москва строить', 'строить атомный', 'атомный авианосец', 'авианосец шторма', 'шторм предполагаться']
['стих', 'поэт', 'доходить домом', 'дом', 'дом сходить']
['режиссер', 'театр', 'создание', 'март белый', 'белый дом']
['россия', 'политика', 'необходимо', 'внешняя политика', 'нво']
['анкара', 'сша', 'турецкую власть', 'турецкий', 'турция']
['ученик школы', 'курганская область', 'пневматический пистолет', 'школа', 'пострадать']
['страна', 'поставка', 'россия', 'экспорт', 'нефть']
['матч', 'высокий', 'шайба', 'сборная', 'команда']
['владимир', 'статуя', 'площадь', 'москва', 'святой']
['перевод', 'классическая поэзия', 'поэзия прозы', 'поэзия', 'бурятский язык']
['клетка', 'миндалина', 'клетка миндалины', 'нервный', 'белок']
['выборы', 'кандидат', 'президентский выборы', 'подмосковье', 'партия']
['газ', 'млрд куба', 'страна', 

#### 3. Ключевые слова и биграммы по частотности и без глаголов

In [34]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [35]:
def get_keywords_bigrams_most_frequent_without_verbs(text, num_keywords):
    """
    берет текст в виде списка слов и количество ключевых слов для извлечения,
    возвращает в качестве ключевых слов самые частотные биграммы текста, 
    согласованные грамматически и не содержащие глаголы, и самые частотные слова, 
    не являющиеся глаголами, в зависимости от их нормированной частотности
    """
    bigrams = [(text[i], text[i + 1]) for i, word in enumerate(text) if i != (len(text) - 1)]
    freq_uninflected_bigrams_numbers = dict([bigram_freq for bigram_freq in Counter(bigrams).most_common()])
    freq_words_numbers = dict([(word_freq[0], word_freq[1] / 2) for word_freq in Counter(text).most_common()])
    freq_items_numbers = list({**freq_uninflected_bigrams_numbers , **freq_words_numbers}.items())
    items_to_remove = []
    for item in freq_items_numbers:
        if type(item[0]) == str:
            word = morph.parse(item[0])[0]
            if 'INFN' in word.tag:
                items_to_remove.append(item)
        else: 
            first_word = morph.parse(item[0][0])[0]
            second_word = morph.parse(item[0][1])[0]
            if 'INFN' in first_word.tag or 'INFN' in second_word.tag:
                items_to_remove.append(item)
    freq_items_numbers = [item for item in freq_items_numbers if item not in items_to_remove]
    freq_items_numbers = sorted(freq_items_numbers, key=lambda item: item[1], reverse=True)[:num_keywords]
    keywords = []
    for item in freq_items_numbers:
        if type(item[0]) == tuple: 
            new_bigram = None
            first_word = morph.parse(item[0][0])[0]
            second_word = morph.parse(item[0][1])[0]
            if 'ADJF' in first_word.tag or 'ADJS' in first_word.tag:
                if second_word.tag.gender:
                    new_bigram = (first_word.inflect({second_word.tag.gender}).word, item[0][1])
            elif 'NOUN' in first_word.tag and 'NOUN' in second_word.tag:
                if second_word.inflect({'gent'}):
                    if 'Surn' not in first_word.tag and 'Name' not in first_word.tag:
                        new_bigram = (item[0][0], second_word.inflect({'gent'}).word)
            if new_bigram:
                keywords.append(' '.join(new_bigram))
            else:
                keywords.append(' '.join(item[0]))
        else:
             keywords.append(item[0]) 
    return keywords

In [36]:
for text in pymorphy2_lemmatized_texts:
    print(get_keywords_bigrams_most_frequent_without_verbs(text, 5))

['ольга васильев', 'исторический источник', 'фгос', 'стандарт', 'источник']
['красота', 'глаз', 'отчаяние', 'порыв', 'порыв отчаяния']
['пепеляев', 'юзефович', 'книга', 'якутия', 'леонид юзефович']
['гонка', 'команда', 'пилот', 'ferrari', 'mclaren']
['есенин', 'поэт', 'клюев', 'смерть есенина', 'смерть']
['медицина', 'медицинский', 'кафедра', 'выпускник', 'практическую медицина']
['островного говор', 'книга', 'русский', 'русская островной', 'мозг']
['ирак', 'война', 'американец', 'партизанский', 'войско']
['нейросеть', 'ai', 'искусственный интеллект', 'клетка', 'искусственный']
['кристалина георгиев', 'россия', 'вступление', 'кристалина', 'георгиев']
['фильм', 'приз', 'картина', 'кино', 'например']
['каша', 'исконно русский', 'столовая', 'рок', 'талон']
['место', 'князь андрея', 'псков', 'город', 'событийный туризм']
['индонезия', 'страна', 'экстремизм терроризма', 'мусульманская община', 'мусульманский']
['игра', 'большой', 'друг', 'парень', 'саша']
['выборы', 'путин', 'президент', 'в

['выставка', 'новый иерусалим', 'кустодиевый', 'музей', 'портрет']
['автомобиль', 'техническая реальность', 'человеческий', 'техносфера', 'технический']
['гонка', 'себастьян феттель', 'пилот', 'ход гонки', 'toro roso']
['джонс', 'клив джонс', 'харви милка', 'право', 'сценарист харви']
['мвф', 'правительство', 'пакет', 'европейский', 'млрд евро']
['уг волан', 'роба хаффа', 'квалификация', 'гонка', 'волан']
['гто', 'гиря', 'чиновник', 'губернатор', 'издалёка весна']
['баттон', 'дженсон баттона', 'рон деннис', 'дженсон', 'команда']
['парниковый газ', 'выброс парниковый', 'выброс', 'газ', 'парниковый']
['миссия', 'миротворческая миссия', 'миротворческий', 'миротворец', 'киев']
['военный', 'военная область', 'область', 'правительство российский', 'российская федерация']
['гайморит', 'нос', 'пазуха', 'придаточная пазуха', 'острый гайморит']
['третьяк', 'стих', 'книга', 'маяковский', 'стихотворение']
['турция', 'синагога', 'теракт', 'еврейская община', 'бейт исраэля']
['машина', 'страна', 'ст

['красная армия', 'армия', 'военная безопасность', 'красный', 'военный']
['украина', 'крым', 'выборы', 'россия', 'российский выборы']
['книга', 'старообрядец', 'автор', 'старовер', 'читатель']
['совет директора', 'совет', 'твц', 'директор', 'канал']
['страна', 'данные', 'развитие', 'будущее', 'проблема']
['химическое оружие', 'россия', 'глава мид', 'рф', 'москва']
['жюля бьянка', 'авария', 'бьянка', 'машина', 'трасса']
['квалификация', 'заезд', 'пеп ориол', 'свободный заезд', 'друг друга']
['береговой охрана', 'береговой', 'охрана', 'арктический', 'канада']
['бурджанадзе', 'саакашвили', 'грузия', 'выборы', 'нино бурджанадзе']
['батальон востока', 'минобороны рф', 'рамзан кадыров', 'сулим ямадаев', 'восток']
['табаков', 'мила одегов', 'божья воля', 'олег табаков', 'движение божий']
['книга', 'половина тайком', 'книга библиотеки', 'библиотека предвоенный', 'предвоенный подмосковный']
['программа', 'фильм', 'главная программа', 'конкурс', 'главный']
['полиция', 'штраф', 'отдел полиции', '

['виктор серж', 'серж', 'виктор', 'революция', 'русский']
['гроссмейстер', 'владимир либерзона', 'шахматный', 'владимир', 'либерзона']
['башар асад', 'исламское государство', 'ирак', 'западная коалиция', 'сторона']
['митинг', 'навальный', 'новость купчиный', 'сторонник', 'глава района']
['население', 'тыс', 'крым', 'полуостров', 'млн руб']
['журналист', 'союз журналиста', 'кадыров', 'рамзан кадыров', 'журналист россии']
['война', 'гибридная война', 'конфликт', 'военный', 'стратегия']
['ребёнок', 'детский сад', 'пиво', 'сад', 'дом']
['савченко', 'верховная рада', 'депутат', 'юрий луценко', 'верховный']
['балет', 'петипа', 'русский', 'русский балет', 'театр']
['центр желания', 'биомагнитное поле', 'магнитное поле', 'головного мозг', 'центр']
['пепеляев', 'зимняя дорога', 'история', 'роман', 'война']
['рекомбинантный инсулин', 'инсулин', 'клетка', 'гормон', 'атака аутоиммунный']
['млрд долл', 'долл', 'еврооблигация общий', 'общая сумма', 'сумма млрд']
['программа', 'фильм', 'фестиваль', '

['гесс', 'степном волк', 'герман', 'роман', 'герман гесс']
['вооружение', 'гиперзвуковой', 'гиперзвуковая ракета', 'сша', 'трамп']
['кузнецов', 'литературный институт', 'юрий кузнецов', 'поэт', 'литературный']
['шпаликов', 'геннадий шпаликов', 'книга шпаликова', 'книга', 'кулагин']
['великий пост', 'пост северокавказский', 'северокавказский железный', 'железная дорога', 'дорога пассажира']
['участник', 'правильный ответ', 'лабораторный', 'ответ', 'роман авдеева']
['русский', 'нецензурное слово', 'роль русский', 'русский бандит', 'техасский полицейский']
['книга', 'роман', 'мир', 'перевод', 'стихотворение']
['улица', 'город', 'москва', 'место', 'горожанин']
['страна', 'япония', 'партия', 'японский', 'лдп']
['данные', 'информационная технология', 'информационный', 'технология', 'новый']
['вода', 'сточная вода', 'система', 'закрытый водооборот', 'завод']
['релятивистская термодинамика', 'релятивистский', 'скорость', 'термодинамика', 'тепло']
['школа', 'ребёнок', 'родитель', 'учитель', 'се

...

['оао кузнецова', 'олимпийский', 'чаша олимпийский', 'олимпийский огонь', 'олимпийская игра']
['млрд евро', 'млрд', 'евро', 'брюссель', 'ципраса']
['экологический', 'алюминий', 'производство', 'экологический рюкзак', 'русало']
['святого патрик', 'святой', 'патрик', 'ирландский', 'зелёный']
['ес', 'вопрос', 'реформа ес', 'данный вопрос', 'дело']
['стильный', 'российский спортсмен', 'стильный чистый', 'арабский эмират', 'бюффон']
['российский военный', 'военный самолёт', 'сирия', 'россия', 'военный']
['тыс долл', 'долл', 'бетси', 'млн долл', 'блуминдейл']
['концерт', 'выборы', 'избирательный участок', 'горбунов', 'молодого избиратель']
['долг', 'руб', 'неплательщик', 'услуга', 'услуга жкх']
['игра', 'церемония закрытия', 'олимпиада', 'сочи', 'игра сочи']
['клетка', 'кожа', 'ген ламинина', 'ген', 'слой']
['редактор', 'техническая наука', 'слово', 'писатель', 'русский язык']
['страна', 'россия', 'природная среда', 'экологический', 'земля']
['продолжение сотрудничества', 'сотрудничество пак

['nautilus pompilius', 'стих', 'кормилец', 'текст', 'интервью']
['квалификация', 'пилот', 'часть заезда', 'штраф', 'часть']
['троцкий', 'ленин', 'чернявский', 'революция', 'георгий чернявский']
['смерть', 'адвокат бывший', 'бывший замгендиректор', 'замгендиректор аэрофлота', 'аэрофлот соратника']
['система', 'зона', 'безопасность', 'система безопасности', 'посетитель']
['олег', 'история', 'корабль', 'война', 'колесо']
['дачный посёлок', 'аэропорт', 'дачный', 'посёлок', 'взлётно-посадочная полоса']
['клоп', 'мраморный клоп', 'азиатский щитник', 'halyomorpha halys', 'грузия']
['toro roso', 'mercedes кий', 'кий райкконный', 'ряд ferrari', 'себастьян феттель']
['тыс долл', 'долл', 'бетси', 'млн долл', 'блуминдейл']
['новый', 'название', 'новый вид', 'название новый', 'оружие']
['парниковый газ', 'выброс парниковый', 'выброс', 'развитие', 'газ']
['южная осетия', 'абхазия южный', 'грузия', 'абхазия', 'южный']
['страна', 'бен ладен', 'свободная страна', 'буш', 'американский']
['орб-2', 'рамза

['здравоохранение', 'медицинский', 'медицинская помощь', 'помощь', 'возможность получения']
['город', 'горожанин', 'зелёный', 'парка', 'самый']
['знание', 'инженерный', 'образование', 'технология', 'инженерная школа']
['команда', 'гонка', 'трасса', 'феттель', 'себастьян феттель']
['ночлежка', 'вопрос', 'нехорошая вещь', 'джерри', 'ребёнок']
['производство', 'алюминий', 'производство алюминия', 'экологический', 'технология']
['управление', 'мо', 'военный', 'система', 'министр обороны']
['гонка', 'джеймс томпсон', 'трасса', 'пилот', 'джеймс']
['александр жуков', 'жуков', 'экономика', 'вто', 'страна']
['благодатный огонь', 'огонь', 'великая суббота', 'русский духовный', 'духовную миссия']
['россия', 'вопрос', 'вступление россии', 'россия вто', 'вто']
['сердце', 'остановка сердца', 'внезапная остановка', 'спортсмен', 'остановка']
['ввс сша', 'млрд долл', 'ввс', 'космический', 'сша']
['газ', 'млрд куба', 'млрд', 'куб', 'страна']
['конструктор', 'главный конструктор', 'главный', 'решение', '

['война', 'ирак', 'американец', 'сша', 'американский']
['мид рф', 'грузия', 'грузинская сторона', 'мид', 'рф']
['гринвуд', 'повесть', 'автор книги', 'история', 'писатель']
['гостиница', 'андрей тарасенко', 'японский', 'компания', 'отель']
['национальный', 'национальная дружина', 'правоохранительный орган', 'национальный корпус', 'дружина']
['стих', 'стих рассказа', 'рассказ', 'шенкман', 'тишина']
['сша', 'встреча', 'канадский', 'канада', 'президент']
['иран', 'белый дом', 'тиллерсон', 'свпд', 'трамп']
['грудинин', 'неделя', 'прошедшая неделя', 'минувшая неделя', 'избирательное законодательство']
['шифровальная машина', 'американский', 'уолкер', 'военно-морской сила', 'вмс сша']
['постмодернистский', 'мир', 'герой', 'постмодернистский герой', 'текст']
['ковч', 'ковча', 'лагерь смерти', 'узник', 'священник']
['петипа', 'балет', 'танец', 'образ', 'театр']
['польский', 'украинский', 'украина', 'польша', 'решение']
['млрд долл', 'бумага', 'выпуск', 'млрд', 'новый']
['президент владимира', '

['смертность', 'рак', 'смертность рака', 'потребление алкоголя', 'рак лёгкий']
['газ', 'газовая опек', 'газовый', 'создание газовый', 'организация']
['валерий рашкина', 'кпрф', 'горком кпрф', 'рашкина', 'московский горком']
['москва', 'активное долголетие', 'старший поколения', 'проект', 'программа']
['акция', 'новосибирский', 'партия', 'яблоко', 'новосибирск']
['криптовалюта', 'цифровой валюта', 'страна', 'крипторубль', 'биткоина самый']
['ес', 'страна', 'орбан', 'политика', 'европа']
['парниковый газ', 'выброс парниковый', 'выброс', 'развитие', 'газ']
['коваль', 'юрий', 'юрий коваль', 'юрий иосич', 'писатель']
['фильм', 'дух огня', 'режиссёр', 'дебютная лента', 'дмитрий фалькович']
['информационный', 'информационная война', 'война', 'информационная безопасность', 'безопасность']


In [37]:
for text in mystem_lemmatized_texts:
    print(get_keywords_bigrams_most_frequent_without_verbs(text, 5))

['исторический источник', 'ольга васильев', 'фгос', 'стандарт', 'источник']
['красота', 'глаз', 'отчаяние', 'порыв', 'порыв отчаяния']
['пепеляев', 'юзефович', 'книга', 'якутия', 'белый']
['команда', 'гонка', 'пилот', 'ким райкконен', 'Ferrari']
['есенин', 'поэт', 'клюев', 'смерть есенина', 'смерть']
['медицина', 'медицинский', 'кафедра', 'выпускник', 'практическую медицина']
['островного говор', 'книга', 'русский', 'русская островной', 'мозг']
['ирак', 'война', 'американец', 'партизанский', 'войско']
['нейросеть', 'искусственный интеллект', 'клетка', 'искусственный', 'интеллект']
['кристалина георгиев', 'россия', 'вступление', 'кристалина', 'вто']
['фильм', 'приз', 'картина', 'кино', 'например']
['каша', 'исконно русский', 'столовая', 'рок', 'талон']
['место', 'князь андрея', 'псков', 'город', 'событийный туризм']
['индонезия', 'страна', 'экстремизм терроризма', 'мусульманская община', 'мусульманский']
['игра', 'друг', 'парень', 'саша', 'зритель']
['выборы', 'путин', 'президент', 'выб

['автомобиль', 'техническая реальность', 'человеческий', 'техносфера', 'технический']
['себастьян феттель', 'гонка', 'пилот', 'Toro Roso', 'гонка нико']
['джонс', 'клив джонс', 'харви милка', 'движение', 'право']
['мвф', 'правительство', 'пакет', 'европейский', 'млрд евро']
['Уг волан', 'роба хафф', 'квалификация', 'гонка', 'волан']
['гто', 'гиря', 'чиновник', 'губернатор', 'издалека весна']
['баттон', 'дженсон баттона', 'рон деннис', 'дженсон', 'команда']
['парниковый газ', 'выброс парниковый', 'выброс', 'газ', 'парниковый']
['миссия', 'миротворческая миссия', 'миротворческий', 'миротворец', 'киев']
['военный', 'военная область', 'область', 'правительство российский', 'российская федерация']
['гайморит', 'нос', 'пазуха', 'придаточная пазуха', 'острый гайморит']
['третьяков', 'стих', 'книга', 'стихотворение', 'маяковский']
['турция', 'синагога', 'теракт', 'среди погибший', 'еврейская община']
['машина', 'страна', 'страховой', 'ремонт', 'дорога']
['память', 'предлобная кора', 'ия', 'лаз

['данные', 'страна', 'развитие', 'проблема', 'будущее']
['химическое оружие', 'россия', 'глава мид', 'рф', 'москва']
['жюль бьянка', 'авария', 'бьянка', 'машина', 'трасса']
['квалификация', 'заезд', 'поул позиции', 'пепе ориола', 'свободный заезд']
['береговой охрана', 'береговой', 'охрана', 'арктический', 'канада']
['бурджанадзе', 'саакашвили', 'грузия', 'выборы', 'нино бурджанадзе']
['батальон востока', 'минобороны рф', 'рамзан кадыров', 'восток', 'батальон']
['табаков', 'милый одегов', 'божья воля', 'олег табаков', 'движение божий']
['книга', 'книжка', 'половина тайком', 'книга библиотеки', 'библиотека предвоенный']
['программа', 'фильм', 'главная программа', 'основной конкурс', 'конкурс']
['полиция', 'штраф', 'отдел полиции', 'тыс руб', 'активист']
['ор', 'ходорковский', 'нга', 'яблоко', 'партия']
['шельф', 'граница', 'континентальный шельф', 'россия', 'континентальный']
['боевого действие', 'су', 'всу', 'авиабаза хмеймити', 'морского пехотинец']
['герман греф', 'бо силай', 'андрей

['война', 'гибридная война', 'военный', 'конфликт', 'стратегия']
['ребенок', 'детский сад', 'пиво', 'сад', 'дом']
['савченко', 'верховный рад', 'депутат', 'юрий луценко', 'верховный']
['балет', 'петипа', 'русский', 'мариус петипа', 'русский балет']
['центр желания', 'биомагнитное поле', 'магнитное поле', 'головного мозг', 'центр']
['пепеляев', 'зимняя дорога', 'история', 'роман', 'война']
['клетка', 'бета клетки', 'рекомбинантный инсулин', 'инсулин', 'гормон']
['млрд долл', 'долл', 'еврооблигация общий', 'общая сумма', 'сумма млрд']
['программа', 'фильм', 'фестиваль', 'й фестиваль', 'фестиваль анимации']
['цвет', 'елена', 'гардероб', 'имидж консультанта', 'мир елены']
['форум', 'россия', 'оппозиция', 'открытая россия', 'активист']
['квартира', 'жилье', 'военнослужащий', 'очередь', 'бесквартирный']
['путин', 'выборы', 'результат', 'москва', 'холодная война']
['рыба', 'потребление рыбы', 'кг', 'тыс', 'потребление']
['плакат', 'павел грудинин', 'улица владивостока', 'рэп исполнителя', 'ма

['книга', 'роман', 'мир', 'перевод', 'стихотворение']
['улица', 'город', 'москва', 'место', 'парк']
['страна', 'япония', 'партия', 'японский', 'бизнес']
['информационный', 'данные', 'информационная технология', 'технология', 'новый']
['вода', 'сточная вода', 'система', 'завод', 'очистка']
['релятивистская термодинамика', 'релятивистский', 'скорость', 'термодинамика', 'тепло']
['школа', 'ребенок', 'родитель', 'самое дело', 'граница дозволенный']
['город', 'тыс км', 'россия', 'сеть', 'выброс']
['экспедиция', 'хребет ломоносова', 'исследование', 'россия', 'никакого отношение']
['евкуров', 'карадаг', 'россия', 'ингушетия', 'мусульманин']
['фос', 'система', 'сооружение', 'сточная вода', 'растение']
['партия', 'проект', 'реформа', 'яблоко', 'власть']
['президент', 'политический', 'атамбаева', 'киргизский', 'власть']
['узбекистан', 'теракт', 'организация', 'хизба ута', 'исламский']
['дом', 'самолет', 'лайнер', 'больница', 'данные']
['волшебный замок', 'бруклин пирса', 'фильм', 'шон бейкер', '

...

['км часа', 'км', 'скорость км', 'поезд', 'час']
['региональный', 'россия', 'власть', 'москва', 'российская сторона']
['путин', 'япония', 'абэ', 'остров', 'японский']
['вода', 'сточная вода', 'система', 'завод', 'очистка']
['страна', 'боец воды', 'вода конца', 'конец марганца', 'текущее расстройство']
['жена волшебника', 'платон', 'дурак', 'жена', 'волшебник']
['камень', 'башня', 'берег', 'кусок', 'белый']
['закон', 'вооружённый конфликт', 'вооруженный', 'россия', 'закон реинтеграции']
['свинья', 'душераздирающий вздох', 'покупка покупки', 'свинья свиньи', 'дело']
['вто', 'азербайджан', 'процесс вступления', 'вступление вто', 'вто определенный']
['советская оккупация', 'млрд долл', 'ущерб', 'советский', 'эстония']
['русский музей', 'музей', 'картина', 'музей людвига', 'выставка']
['закупка', 'поставщик', 'млрд руб', 'росат', 'система закупки']
['выпускник школы', 'школа крыма', 'уровень подготовки', 'форма', 'законопроект']
['управление', 'мо', 'военный', 'система', 'министр обороны']


['цирконий', 'млрд руб', 'металл', 'производство', 'материал']
['член экипажа', 'сухогруз', 'экипаж', 'керченский пролив', 'экипаж сухогруза']
['единая россия', 'россия', 'смешанная система', 'россия депутата', 'депутат']
['патриарх', 'патриарх кирилла', 'март', 'кирилл', 'болгария']
['путин', 'япония', 'абэ', 'остров', 'японский']
['пенсионный', 'пенсионный возраст', 'возраст', 'цифровой экономика', 'пенсионная система']
['киргизия', 'страна', 'жээнбеков', 'йакын инкар', 'персидский залив']
['проект', 'тонна экв', 'млн тонны', 'сокращение выброса', 'тонна']
['еврейский', 'власть', 'гойтейн', 'гениза', 'документ']
['км ч', 'глава комитета', 'комитет госдумы', 'госдума транспорта', 'транспорт строительства']
['единая россия', 'россия', 'единый', 'партия', 'страна']
['врач', 'московский врач', 'тыс руб', 'медицинский', 'руб']
['ходорковский', 'путин', 'экс главы', 'глава юкоса', 'россия']
['поэт', 'клуб', 'алкогольная тематика', 'поэт игоря', 'член клуба']
['кафе поэта', 'винный бар', 'б

['фильм', 'герой', 'роль', 'безруков', 'сергей безруков']
['дом', 'наташа', 'глеб', 'серафим', 'что-то']
['избирательное законодательство', 'законодательство рф', 'возможность создания', 'создание предвыборный', 'предвыборный блок']
['россия', 'скрипаль', 'нато', 'соединенный королевство', 'нга']
['организация', 'список', 'саудовская аравия', 'террористический', 'террористическая организация']
['Volvo', 'новый', 'Syan Racing', 'автомобиль Volvo', 'чемпионат']
['турция', 'отношение анкары', 'анкара вашингтона', 'точка кипения', 'отношение']
['бпло', 'система', 'компания AeroVironment', 'аппарат', 'сша']
['македония', 'православная церковь', 'греция', 'страна', 'слово македонии']
['афганистан', 'страна', 'антиправительственная группировка', 'группировка', 'власть']
['отец', 'василий', 'княжество', 'престол', 'сын']
['река агура', 'вчера президент', 'президент рф', 'рф владимира', 'владимир путин']
['город африн', 'африн', 'агентство', 'город', 'президент турции']
['строительство', 'олимп

['страна', 'атомная энергетика', 'реактор', 'энергетика', 'энергетический']
['фильм', 'чёрная пантера', 'кратко', 'главный', 'черный']
['компания', 'оценщик', 'конкурс', 'земля', 'слово']
['папа', 'франциск', 'кардинал', 'папа франциска', 'епископ']
['поэт', 'вечер поэта', 'поэт виктора', 'виктор коллегорский', 'ангельское пение']
['сша', 'американский', 'спецслужба', 'война', 'мировой война']
['вид', 'млн', 'экологический', 'чужеродный вид', 'земля']
['наука', 'ран', 'академия науки', 'академия', 'президент ран']
['тыс руб', 'тыс', 'зарплата', 'руб', 'финансовый сектор']
['оценка технический', 'технический комитет', 'комитет опек+', 'опек+ уровень', 'уровень соблюдения']
['автомобиль', 'техническая реальность', 'человеческий', 'техносфера', 'технический']
['крокодил', 'разведение крокодила', 'новый', 'сторона', 'товарищ']
['фильм', 'млн руб', 'кино', 'проект', 'млн']
['байкал', 'вода', 'иркутская гэс', 'уровень', 'гэс']
['тысяча рубля', 'рубль', 'тысяча', 'издание', 'раритетное издани

['руслан', 'нии', 'секретарь парткома', 'интернат', 'молодой']
['театр', 'режиссер', 'юрий бутусов', 'худрук', 'директор']
['ядерный', 'ядерное оружие', 'иран', 'оружие', 'саудовская аравия']
['команда', 'пилот', 'Red Bull', 'льюис хэмилтон', 'хэмилтон']
['елка', 'игрушка', 'коробка', 'елочка', 'стеклянный']
['российский авиадиспетчер', 'обвинение', 'провоцирование авиакатастрофы', 'авиакатастрофа лайнера', 'лайнер президента']
['россия', 'страна', 'власть', 'власть общества', 'общество']
['туркмения', 'спутниковое телевидение', 'российский', 'туркменский', 'телевидение']
['греция', 'план', 'новый план', 'ципрас', 'новый']
['сирия', 'российский', 'президент', 'российский военный', 'американское издание']
['страна', 'асеан', 'россия', 'соглашение', 'страна асеан']
['керченский пролив', 'тонна', 'сера', 'порт кавказа', 'мчс россии']
['океан', 'мирового океан', 'мировой', 'пластиковый', 'тихий океан']
['сергей скрипаль', 'россия', 'неоспоримое доказательство', 'россия отравления', 'экс по

#### 4. Ключевые слова по TF-IDF

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [39]:
def get_keywords_tf_idf(texts, num_keywords):
    """
    берет список текстов в виде строк или списков
    и количество ключевых слов для извлечения,
    возвращает в качестве ключевых слов слова, отобранные методом TF-IDF
    """
    keywords = []
    make_tf_idf = TfidfVectorizer()
    for text in texts:
        if type(text) == list:
            texts_as_tfidf_vectors = make_tf_idf.fit_transform(' '.join(text) for text in texts)
        elif type(text) == str:
            texts_as_tfidf_vectors = make_tf_idf.fit_transform(texts)
    id2word = {i:word for i, word in enumerate(make_tf_idf.get_feature_names())} 
    for text_row in range(texts_as_tfidf_vectors.shape[0]): 
        row_data = texts_as_tfidf_vectors.getrow(text_row) 
        words_for_this_text = row_data.toarray().argsort() 
        top_words_for_this_text = words_for_this_text [0,:-(num_keywords+1):-1] 
        keywords_for_this_text = [id2word[w] for w in top_words_for_this_text]
        keywords.append(keywords_for_this_text)
    return keywords

In [41]:
for text_keywords in get_keywords_tf_idf(pymorphy2_lemmatized_texts, 5):
    print(text_keywords)

['фгос', 'стандарт', 'васильев', 'образовательный', 'ольга']
['красота', 'отчаяние', 'порыв', 'кошка', 'глаз']
['пепеляев', 'юзефович', 'якутия', 'восстание', 'книга']
['гонка', 'команда', 'mclaren', 'пилот', 'ferrari']
['есенин', 'клюев', 'поэт', 'мариенгоф', 'борода']
['медицина', 'кафедра', 'выпускник', 'медицинский', 'рудна']
['говор', 'островной', 'анатомия', 'мозг', 'книга']
['ирак', 'партизанский', 'американец', 'хусейн', 'война']
['ai', 'нейросеть', 'клетка', 'пиксель', 'dl']
['кристалина', 'георгиев', 'вто', 'вступление', 'вб']
['приз', 'фильм', 'кино', 'берлинал', 'медведь']
['каша', 'столовая', 'талон', 'фиалка', 'унитаз']
['псков', 'ганзейский', 'туризм', 'ухаб', 'князь']
['индонезия', 'экстремизм', 'мусульманский', 'терроризм', 'взрыв']
['стример', 'youtube', 'игра', 'парень', 'саша']
['выборы', 'путин', 'признавать', 'демократический', 'мнв']
['водный', 'пдк', 'вода', 'загрязнение', 'очистка']
['школа', 'образование', 'школьник', 'олимпиада', 'столичный']
['фильм', 'содер

In [42]:
for text_keywords in get_keywords_tf_idf(mystem_lemmatized_texts, 5):
    print(text_keywords)

['фгос', 'стандарт', 'васильев', 'образовательный', 'ольга']
['красота', 'отчаяние', 'порыв', 'кошка', 'глаз']
['пепеляев', 'юзефович', 'якутия', 'восстание', 'книга']
['гонка', 'команда', 'mclaren', 'пилот', 'ferrari']
['есенин', 'клюев', 'поэт', 'мариенгоф', 'борода']
['медицина', 'кафедра', 'выпускник', 'медицинский', 'рудн']
['говор', 'островной', 'мозг', 'анатомия', 'книга']
['ирак', 'партизанский', 'бадр', 'американец', 'хусейн']
['нейросеть', 'клетка', 'пиксел', 'интеллект', 'подобно']
['кристалина', 'георгиев', 'вто', 'вступление', 'вб']
['приз', 'фильм', 'кино', 'берлинале', 'медведь']
['каша', 'талон', 'фиалка', 'столовая', 'унитаз']
['псков', 'ганзейский', 'туризм', 'ухаб', 'князь']
['индонезия', 'экстремизм', 'мусульманский', 'терроризм', 'взрыв']
['стример', 'ютюбер', 'youtube', 'игра', 'парень']
['выборы', 'путин', 'признавать', 'поздравлять', 'демократический']
['водный', 'вода', 'загрязнение', 'сток', 'очистка']
['школа', 'образование', 'школьник', 'олимпиада', 'столичн

#### 5. Ключевые слова и биграммы по TF-IDF

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
from stop_words import get_stop_words
from pymystem3 import Mystem
m = Mystem()

In [44]:
rus_stop_words = get_stop_words('ru')
rus_stop_words.extend(['чей', 'свой', 'ежели', 'нешто', 'из-за'])

In [45]:
def get_preprocessed_text(text):
    """
    берет текст в виде строки и возвращает его без стоп-слов и
    лемматизированным pymorphy2 в виде списка слов
    """
    preprocessed_text = ' '.join([morph.parse(word)[0].normal_form.strip(punctuation+'.— ') 
                       for word in word_tokenize(text)
                       if re.search('[а-яёa-z]', word, flags=re.I)])
    return preprocessed_text

In [46]:
def get_keywords_bigrams_tf_idf(texts, num_keywords):
    """
    берет список не лемматизированных текстов в виде строк
    и количество ключевых слов для извлечения,
    возвращает в качестве ключевых слов слова и биграммы, 
    отобранные методом TF-IDF
    """
    keywords = []
    make_tf_idf = TfidfVectorizer(stop_words=rus_stop_words, ngram_range=(1, 2))
    texts_as_tfidf_vectors = make_tf_idf.fit_transform(get_preprocessed_text(text) for text in texts)
    id2word = {i : word for i, word in enumerate(make_tf_idf.get_feature_names())} 
    for text_row in range(texts_as_tfidf_vectors.shape[0]): 
        row_data = texts_as_tfidf_vectors.getrow(text_row) 
        words_for_this_text = row_data.toarray().argsort() 
        top_words_for_this_text = words_for_this_text [0, : -(num_keywords + 1) : -1] 
        keywords_for_this_text = [id2word[w] for w in top_words_for_this_text]
        keywords.append(keywords_for_this_text)
    return keywords

In [47]:
for text_keywords in get_keywords_bigrams_tf_idf(texts, 5):
    print(text_keywords)

['фгос', 'исторический источник', 'ольга васильев', 'стандарт', 'васильев']
['красота', 'отчаяние', 'порыв', 'кошка', 'глаз']
['пепеляев', 'юзефович', 'якутия', 'восстание', 'книга']
['гонка', 'команда', 'mclaren', 'пилот', 'ferrari']
['есенин', 'клюев', 'поэт', 'мариенгоф', 'борода']
['медицина', 'кафедра', 'выпускник', 'медицинский', 'ординатор']
['говор', 'островной говор', 'островной', 'русский островной', 'анатомия']
['ирак', 'партизанский', 'американец', 'хусейн', 'война']
['ai', 'нейросеть', 'клетка', 'dl', 'пиксель']
['кристалина георгиев', 'кристалина', 'георгиев', 'вто', 'вступление']
['приз', 'фильм', 'кино', 'берлинал', 'медведь']
['каша', 'столовая', 'талон', 'фиалка', 'унитаз']
['псков', 'князь андрей', 'ганзейский', 'туризм', 'ухаб']
['индонезия', 'экстремизм', 'мусульманский', 'экстремизм терроризм', 'терроризм']
['стример', 'youtube', 'игра', 'парень', 'саша']
['выборы', 'признавать выборы', 'путин', 'признавать', 'демократический']
['водный', 'пдк', 'водный объект', '

#### 6. Ключевые слова по degree centrality графа совместной встречаемости слов в тексте

In [50]:
import networkx as nx

In [51]:
def get_keywords_graph_degree_centrality(text, num_keywords):
    """
    берет текст в виде списка слов и количество ключевых слов для извлечения,
    возвращает в качестве ключевых слов узлы графа с самой высокой степенью degree centrality
    """
    G = nx.Graph()
    G.add_edges_from([(text[i], text[i+1]) for i, item in enumerate(text) if i != (len(text)-1)])
    dc = nx.degree_centrality(G)
    keywords = [node for node in sorted(dc, key=dc.get, reverse=True)[:num_keywords]]
    return keywords

In [52]:
for text in pymorphy2_lemmatized_texts:
    print(get_keywords_graph_degree_centrality(text, 5))

['фгос', 'стандарт', 'предмет', 'содержание', 'источник']
['красота', 'отчаяние', 'глаз', 'кошка', 'порыв']
['пепеляев', 'книга', 'юзефович', 'якутия', 'война']
['команда', 'гонка', 'пилот', 'ferrari', 'mclaren']
['есенин', 'поэт', 'клюев', 'сергей', 'мариенгоф']
['медицина', 'медицинский', 'кафедра', 'выпускник', 'работа']
['книга', 'русский', 'мозг', 'глава', 'анатомия']
['ирак', 'война', 'американец', 'партизанский', 'войско']
['нейросеть', 'ai', 'клетка', 'самый', 'основа']
['россия', 'вступление', 'вто', 'банк', 'переговоры']
['фильм', 'приз', 'картина', 'кино', 'например']
['каша', 'сидеть', 'столовая', 'начинать', 'талон']
['место', 'псков', 'город', 'концепция', 'туризм']
['индонезия', 'страна', 'сша', 'взрыв', 'мусульманский']
['игра', 'говорить', 'парень', 'стать', 'большой']
['выборы', 'путин', 'президент', 'сша', 'рф']
['вода', 'водный', 'качество', 'очистка', 'россия']
['образование', 'школа', 'москва', 'школьник', 'международный']
['фильм', 'стать', 'герой', 'содерберг', 

['савченко', 'депутат', 'верховный', 'рада', 'март']
['капнистый', 'василий', 'поэт', 'стать', 'театр']
['война', 'поэтому', 'говорить', 'путин', 'казаться']
['формат', 'февраль', 'рамка', 'глава', 'нормандский']
['город', 'горожанин', 'зелёный', 'парка', 'место']
['мозг', 'стимуляция', 'память', 'помощь', 'расстройство']
['тайник', 'булыжник', 'место', 'американец', 'альфовца']
['депутат', 'среда', 'заседание', 'выборы', 'бюллетень']
['сумароков', 'гамлет', 'писать', 'тредиаковский', 'русский']
['халил', 'кизлярский', 'дагестан', 'иго', 'джихад']
['росатом', 'закупка', 'поставщик', 'корпорация', 'процедура']
['школа', 'ребёнок', 'психолог', 'работа', 'школьный']
['майдан', 'акция', 'саакашвили', 'киев', 'вчера']
['экспедиция', 'арктика', 'ледокол', 'сша', 'канада']
['пг', 'компания', 'инвестор', 'выброс', 'российский']
['ор', 'премия', 'ходорковский', 'журналист', 'организация']
['волна', 'гравитационный', 'физика', 'премия', 'штат']
['российский', 'грузия', 'президент', 'страна', 'ма

['русский', 'книга', 'париж', 'писатель', 'издательство']
['группировка', 'египетский', 'брат-мусульманин', 'египет', 'власть']
['российский', 'образование', 'вто', 'услуга', 'россия']
['проект', 'корабль', 'программа', 'единица', 'вмф']
['пожилой', 'помощь', 'социальный', 'специальный', 'возраст']
['церковный', 'церковь', 'зарубежник', 'рпцз', 'русский']
['горький', 'писатель', 'алексей', 'автор', 'отношение']
['пилот', 'гонка', 'роба', 'хаффа', 'позиция']
['андромеда', 'татуировка', 'исследование', 'россиянин', 'счастливый']
['ячейка', 'солнечный', 'перовскит', 'энергия', 'технология']
['водохранилище', 'вода', 'волга', 'отметка', 'чебоксарский']
['врач', 'специалист', 'уровень', 'главный', 'медицинский']
['вто', 'киев', 'россия', 'киргизия', 'украина']
['telegram', 'фсб', 'суд', 'приказ', 'ключ']
['операция', 'кость', 'оториноларингология', 'ника', 'педиатрия']
['поэт', 'клуб', 'писатель', 'выставка', 'посвятить']
['фильм', 'панахи', 'режиссёр', 'последний', 'роль']
['команда', 'гон

['партия', 'гудков', 'выборы', 'собчак', 'яблоко']
['клетка', 'вода', 'нейрон', 'матрица', 'помощь']
['путин', 'выборы', 'власть', 'избиратель', 'результат']
['этнический', 'народ', 'население', 'страна', 'нация']
['цена', 'нефть', 'руб', 'долл', 'греция']
['индия', 'сша', 'пакистан', 'россия', 'страна']
['вопрос', 'новый', 'представление', 'реакция', 'научный']
['конкурс', 'право', 'произвол', 'знать', 'закон']
['афганистан', 'талибан', 'афганский', 'провинция', 'талиб']
['бояться', 'литр', 'человечество', 'съесть', 'дом']
['словарь', 'электронный', 'бумажный', 'стать', 'новый']
['тележка', 'клиент', 'кофе', 'работа', 'доминик']
['цифровой', 'компания', 'новый', 'проект', 'энергетика']
['религиозный', 'радикальный', 'фсин', 'мусульманин', 'экстремизм']
['создать', 'цивилизация', 'вид', 'существо', 'разумный']
['дагестан', 'республика', 'этнический', 'религиозный', 'даргинец']
['комиссия', 'работа', 'представитель', 'мак', 'место']
['аум', 'асахара', 'ниий', 'теракт', 'суд']
['русский'

['япония', 'путин', 'остров', 'абэ', 'россия']
['возраст', 'пенсионный', 'гражданин', 'экономика', 'условие']
['киргизия', 'страна', 'запретить', 'жээнбеков', 'ислам']
['проект', 'сбербанк', 'выброс', 'минэкономразвития', 'конкурс']
['еврейский', 'власть', 'гойтейн', 'документ', 'община']
['москвичев', 'скорость', 'км/ча', 'дорога', 'комитет']
['россия', 'партия', 'страна', 'единый', 'политический']
['врач', 'медицинский', 'хороший', 'пациент', 'проект']
['ходорковский', 'путин', 'россия', 'политический', 'оппозиция']
['поэт', 'клуб', 'писатель', 'выставка', 'посвятить']
['идти', 'поэт', 'город', 'сей', 'царь']
['сирия', 'рф', 'действие', 'оон', 'дамаск']
['послание', 'задача', 'считать', 'экономика', 'путин']
['россия', 'российский', 'энергетический', 'внешний', 'экономика']
['греция', 'страна', 'новый', 'предложение', 'греческий']
['потепление', 'скотомогильник', 'язва', 'арктический', 'побережье']
['поэт', 'клуб', 'писатель', 'выставка', 'посвятить']
['химический', 'правительственны

['отец', 'василий', 'знать', 'княжество', 'стать']
['карта', 'оплата', 'пассажир', 'автобус', 'мострансавто']
['редактор', 'автор', 'текст', 'сайт', 'редакция']
['взрыв', 'произойти', 'ленинский', 'район', 'махачкала']
['российский', 'рф', 'сидельников', 'великобритания', 'движение']
['сша', 'кндр', 'китай', 'ким', 'корея']
['команда', 'гонка', 'mercedes', 'пилот', 'деннис']
['последний', 'регион', 'область', 'рф', 'доля']
['достоевский', 'победоносцев', 'россия', 'писатель', 'народ']
['слово', 'язык', 'словарь', 'термин', 'понятие']
['трамп', 'помпео', 'тиллерсон', 'президент', 'госсекретарь']
['ммр', 'безопасность', 'финляндия', 'случай', 'стать']
['афганистан', 'страна', 'боевик', 'иго', 'провинция']
['слуцкий', 'дом', 'поэт', 'стих', 'знать']
['ор', 'ходорковский', 'россия', 'обыск', 'заявить']
['россиянин', 'продукт', 'здоровый', 'питание', 'рацион']
['получить', 'ранение', 'район', 'данные', 'республиканский']
['объект', 'вчера', 'путин', 'вайншток', 'слово']
['молодой', 'обществ

In [53]:
for text in mystem_lemmatized_texts:
    print(get_keywords_graph_degree_centrality(text, 5))

['фгос', 'стандарт', 'предмет', 'содержание', 'источник']
['красота', 'отчаяние', 'глаз', 'кошка', 'уходить']
['пепеляев', 'книга', 'юзефович', 'белый', 'якутия']
['команда', 'гонка', 'пилот', 'Ferrari', 'McLaren']
['есенин', 'поэт', 'клюев', 'сергей', 'говорить']
['медицина', 'медицинский', 'кафедра', 'выпускник', 'работа']
['книга', 'русский', 'мозг', 'глава', 'анатомия']
['ирак', 'война', 'американец', 'партизанский', 'войско']
['нейросеть', 'клетка', 'самый', 'интеллект', 'основа']
['россия', 'вступление', 'вто', 'переговоры', 'банк']
['фильм', 'приз', 'картина', 'кино', 'например']
['каша', 'сидеть', 'начинать', 'столовая', 'талон']
['место', 'псков', 'город', 'концепция', 'туризм']
['индонезия', 'страна', 'сша', 'взрыв', 'мусульманский']
['игра', 'становиться', 'говорить', 'парень', 'саша']
['выборы', 'путин', 'президент', 'сша', 'рф']
['вода', 'водный', 'качество', 'очистка', 'сток']
['школа', 'образование', 'москва', 'школьник', 'международный']
['фильм', 'снимать', 'становитьс

['команда', 'гонка', 'титул', 'подиум', 'пилот']
['больница', 'помощь', 'высокотехнологичный', 'метод', 'лечение']
['бронштейн', 'книга', 'маршак', 'матвей', 'теория']
['предприятие', 'завод', 'инвестор', 'условие', 'продажа']
['украина', 'корабль', 'крым', 'предложение', 'киев']
['родитель', 'школа', 'ребенок', 'учитель', 'страна']
['инвалид', 'прокуратура', 'центр', 'чемпионат', 'район']
['навальный', 'акция', 'задерживать', 'проходить', 'полиция']
['борьба', 'коррупция', 'антикоррупционный', 'путин', 'президент']
['парк', 'город', 'горожанин', 'зеленый', 'место']
['национальный', 'ахмат', 'республика', 'статья', 'кадровый']
['вто', 'азербайджан', 'мамедгулиев', 'страна', 'документ']
['аэс', 'атомный', 'страна', 'новый', 'реактор']
['медаль', 'сборная', 'место', 'спортсмен', 'заезд']
['украина', 'украинский', 'закон', 'донбасс', 'февраль']
['кандидат', 'праймериз', 'избиратель', 'проигрывать', 'партия']
['налог', 'компания', 'иностранный', 'нефтяник', 'венесуэла']
['президент', 'новы

['ребенок', 'учитель', 'школа', 'ситуация', 'урок']
['москва', 'столица', 'тыс', 'новый', 'город']
['российский', 'легков', 'россиянин', 'советский', 'экипаж']
['пивоваров', 'мвд', 'акция', 'нга', 'выборы']
['мусор', 'отходы', 'развитие', 'процесс', 'экологический']
['земля', 'олимпиада', 'сочи', 'олимпийский', 'тыс']
['сирия', 'вашингтон', 'дамаск', 'союзник', 'москва']
['гессе', 'герман', 'играть', 'роман', 'писатель']
['вооружение', 'сша', 'трамп', 'россия', 'президент']
['поэт', 'кузнецов', 'творчество', 'юрий', 'читать']
['шпаликов', 'книга', 'друг', 'кулагин', 'фильм']
['постный', 'блюдо', 'вагон-ресторан', 'пост', 'северо']
['участник', 'лабораторный', 'ответ', 'говорить', 'балл']
['говорить', 'знать', 'звонить', 'русский', 'давать']
['книга', 'роман', 'выходить', 'писать', 'мир']
['улица', 'город', 'москва', 'место', 'горожанин']
['страна', 'япония', 'партия', 'японский', 'бизнес']
['данные', 'технология', 'информационный', 'новый', 'проблема']
['вода', 'система', 'очистка', 'з

['рф', 'россия', 'скрипаль', 'королевство', 'новичок']
['глянец', 'новый', 'диссертация', 'россия', 'алена']
['суд', 'дальнобойщик', 'бажутин', 'опра', 'являться']
['кишинев', 'молдавия', 'рф', 'ульяновск', 'консультация']
['тепло', 'геотермальный', 'земля', 'тепловой', 'температура']
['театр', 'табаков', 'роль', 'актер', 'сыграть']
['район', 'стекло', 'стихия', 'ущерб', 'край']
['россия', 'экономика', 'власть', 'китай', 'политический']
['трамп', 'сша', 'отношение', 'становиться', 'россия']
['страна', 'россия', 'общество', 'развитие', 'происходить']
['россия', 'мировой', 'страна', 'мир', 'международный']
['помпео', 'трамп', 'глава', 'президент', 'цру']
['снег', 'кошка', 'метель', 'уходить', 'ветер']
['подкомиссия', 'след', 'взрыв', 'последний', 'расследование']
['эколог', 'организация', 'экологический', 'россия', 'крупный']
['текст', 'нонконформизм', 'объявление', 'номинация', 'выдвижение']
['приднестровье', 'конфликт', 'сторона', 'переговоры', 'молдова']
['говорить', 'путин', 'казатьс

['синай', 'российский', 'рф', 'самолет', 'совещание']
['путин', 'россия', 'власть', 'государство', 'вопрос']
['россия', 'отходы', 'регион', 'российский', 'новый']
['территория', 'сирия', 'дамаск', 'турция', 'район']
['кокаин', 'страна', 'колумбия', 'кг', 'наркотик']
['россия', 'грузия', 'нато', 'сторона', 'грузинский']
['российский', 'американский', 'соцсеть', 'страница', 'Facebook']
['мусор', 'завод', 'отходы', 'диоксид', 'центр']
['вид', 'биосфера', 'планета', 'новый', 'вымирание']
['представитель', 'конгрессмен', 'палата', 'конгресс', 'поддерживать']
['филинков', 'суд', 'изолятор', 'дело', 'арест']
['ученый', 'глобальный', 'планета', 'климат', 'некоторый']
['женщина', 'гаи', 'право', 'вождение', 'сотрудник']
['армия', 'румыния', 'военный', 'нато', 'страна']
['соглашение', 'сша', 'страна', 'сторона', 'торговля']
['саркозить', 'дело', 'ливия', 'кампания', 'каддафи']
['тело', 'интеллект', 'машина', 'новый', 'боль']
['студент', 'футбол', 'мгу', 'команда', 'чемпионат']
['россия', 'минист

['экономический', 'реформатор', 'экономика', 'доход', 'расход']
['рф', 'новичок', 'россия', 'скрипаль', 'великобритания']
['вода', 'рентген', 'рентгеновский', 'плотность', 'ученый']
['книга', 'художник', 'русский', 'княгиня', 'герр']
['мид', 'английский', 'знать', 'посольство', 'говорить']
['греция', 'саммит', 'страна', 'афины', 'кредитор']
['горький', 'артамонов', 'дело', 'роман', 'фабрика']
['страна', 'мировой', 'добыча', 'млн', 'сша']
['иван', 'сообщать', 'фонд', 'аргентинский', 'аргентина']
['фильм', 'теракт', 'бекмамбетов', 'работа', 'картина']
['январь', 'выплата', 'доход', 'реальный', 'февраль']
['арктика', 'музей', 'петербург', 'работа', 'судно']
['афганистан', 'азия', 'регион', 'боевик', 'иго']
['туркменистан', 'бердымухамедов', 'выборы', 'президент', 'туркменский']
['конференция', 'встреча', 'нормандский', 'мюнхенский', 'безопасность']
['савченко', 'депутат', 'верховный', 'власть', 'теракт']
['сша', 'израиль', 'израильский', 'сирия', 'удар']
['су', 'самолет', 'российский', 'с

['бояться', 'литр', 'человечество', 'съедать', 'дом']
['учение', 'россия', 'сша', 'регион', 'каспий']
['вейнингер', 'книга', 'пол', 'женщина', 'война']
['честь', 'военный', 'зал', 'военачальник', 'национальный']
['эндаумент', 'мгимо', 'университет', 'фонд', 'проект']
['украина', 'донбасс', 'киев', 'миротворец', 'волкер']
['грудинин', 'подмосковье', 'кпрф', 'кандидат', 'выборы']
['доклад', 'арктика', 'сша', 'россия', 'ес']
['нобелевский', 'премия', 'номинация', 'философ', 'литературный']
['премия', 'поэт', 'летие', 'нга', 'книга']
['стих', 'разный', 'входить', 'поэт', 'поэзия']
['военный', 'область', 'судан', 'сотрудничество', 'подготовка']
['летие', 'поэт', 'игорь', 'елена', 'александр']
['грузия', 'татунашвили', 'власть', 'переговоры', 'слово']
['становиться', 'мир', 'чиновник', 'книга', 'александр']
['территория', 'район', 'зона', 'использование', 'котельная']
['аспирантура', 'нии', 'аспирант', 'научный', 'подготовка']
['эйзенштейн', 'книга', 'архив', 'музей', 'режиссер']
['иностране

### Оценка способов

In [74]:
import json

In [75]:
manual_keywords = list(data.keywords)

Начинаем сравнивать их с т.н. предсказанными ключевыми словами 

Вначале лемматизируем двумя способами корпус текстов

In [76]:
pymorphy2_lemmatized_corpus_texts = [get_pymorphy2_lemmatized_text(text) for text in texts]

In [77]:
mystem_lemmatized_corpus_texts = [get_mystem_lemmatized_text(text) for text in texts]

In [88]:
def get_all_methods_keywords(num_keywords):
    """
    берет количество запрашиваемых ключевых слов в наборе
    и возвращает списки наборов ключевых слов
    """
    #1
    predicted_keywords_bigrams_tf_idf = get_keywords_bigrams_tf_idf(texts, num_keywords)
    #2
    pymorphy2_predicted_keywords_bigrams_most_frequent = [get_keywords_bigrams_most_frequent(text, num_keywords)
                                                          for text in pymorphy2_lemmatized_corpus_texts]
    #3
    pymorphy2_predicted_keywords_bigrams_most_frequent_without_verbs = [get_keywords_bigrams_most_frequent_without_verbs
                                                                        (text, num_keywords) for text in 
                                                                        pymorphy2_lemmatized_corpus_texts]
    #4
    pymorphy2_predicted_keywords_tf_idf = get_keywords_tf_idf(pymorphy2_lemmatized_corpus_texts, num_keywords)
    #5
    pymorphy2_predicted_keywords_graph_degree_centrality = [get_keywords_graph_degree_centrality(text, num_keywords)
                                                            for text in pymorphy2_lemmatized_corpus_texts]
    #6 
    mystem_predicted_keywords_most_frequent = [get_keywords_most_frequent(text, num_keywords) 
                                               for text in mystem_lemmatized_corpus_texts]
    #7
    mystem_predicted_keywords_bigrams_most_frequent = [get_keywords_bigrams_most_frequent(text, num_keywords) 
                                                       for text in mystem_lemmatized_corpus_texts]
    #8
    mystem_predicted_keywords_bigrams_most_frequent_without_verbs = [get_keywords_bigrams_most_frequent_without_verbs
                                                                     (text, num_keywords)
                                                                     for text in mystem_lemmatized_corpus_texts]
    #9
    mystem_predicted_keywords_tf_idf = get_keywords_tf_idf(mystem_lemmatized_corpus_texts, num_keywords)
    #10
    mystem_predicted_keywords_graph_degree_centrality = [get_keywords_graph_degree_centrality(text, num_keywords) 
                                                         for text in mystem_lemmatized_corpus_texts]
    #11
    pymorphy2_predicted_keywords_most_frequent = [get_keywords_most_frequent(text, num_keywords)
                                                  for text in pymorphy2_lemmatized_corpus_texts]
    return pymorphy2_predicted_keywords_most_frequent, \
           pymorphy2_predicted_keywords_bigrams_most_frequent,\
           pymorphy2_predicted_keywords_bigrams_most_frequent_without_verbs, \
           pymorphy2_predicted_keywords_tf_idf, \
           pymorphy2_predicted_keywords_graph_degree_centrality,\
           mystem_predicted_keywords_most_frequent,\
           mystem_predicted_keywords_bigrams_most_frequent,\
           mystem_predicted_keywords_bigrams_most_frequent_without_verbs,\
           mystem_predicted_keywords_tf_idf,\
           mystem_predicted_keywords_graph_degree_centrality,\
           predicted_keywords_bigrams_tf_idf

In [89]:
predicted_keywords_bigrams_tf_idf, \
pymorphy2_predicted_keywords_bigrams_most_frequent,\
pymorphy2_predicted_keywords_bigrams_most_frequent_without_verbs, \
pymorphy2_predicted_keywords_tf_idf, \
pymorphy2_predicted_keywords_graph_degree_centrality,\
mystem_predicted_keywords_most_frequent,\
mystem_predicted_keywords_bigrams_most_frequent,\
mystem_predicted_keywords_bigrams_most_frequent_without_verbs,\
mystem_predicted_keywords_tf_idf,\
mystem_predicted_keywords_graph_degree_centrality,\
pymorphy2_predicted_keywords_most_frequent = get_all_methods_keywords(12)

Чтобы получить названия вызываемых функций для наглядности рейтинга

In [90]:
from inspect import currentframe

In [91]:
def get_name_of_function_returned_var(var):
    """берет переменную и возвращает название функции, которая ее произвела"""
    callers_local_vars = currentframe().f_back.f_back.f_locals.items()
    name_of_function_returned_var = [var_name for var_name, var_val 
                                     in callers_local_vars if var_val is var][0]
    return name_of_function_returned_var

Функция оценки качества предсказания:

In [96]:
def get_evaluation(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    method_name = get_name_of_function_returned_var(predicted_kws)
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    mean_precision = round(np.mean(precisions), 2)
    mean_recall = round(np.mean(recalls), 2)
    fmeasure = round(np.mean(f1s), 2)
    jaccard_index = round(np.mean(jaccards), 2)
    return method_name, (mean_precision, mean_recall, fmeasure, jaccard_index)

In [99]:
import numpy as np

In [101]:
evaluations = []
#1 
evaluations.append(get_evaluation(manual_keywords, pymorphy2_predicted_keywords_most_frequent))
#2
evaluations.append(get_evaluation(manual_keywords, pymorphy2_predicted_keywords_bigrams_most_frequent))
#3
evaluations.append(get_evaluation(manual_keywords, pymorphy2_predicted_keywords_bigrams_most_frequent_without_verbs))
#4
evaluations.append(get_evaluation(manual_keywords, pymorphy2_predicted_keywords_tf_idf))
#5
evaluations.append(get_evaluation(manual_keywords, pymorphy2_predicted_keywords_graph_degree_centrality))
#6
evaluations.append(get_evaluation(manual_keywords, mystem_predicted_keywords_most_frequent))
#7
evaluations.append(get_evaluation(manual_keywords, mystem_predicted_keywords_bigrams_most_frequent))
#8
evaluations.append(get_evaluation(manual_keywords, mystem_predicted_keywords_bigrams_most_frequent_without_verbs))
#9
evaluations.append(get_evaluation(manual_keywords, mystem_predicted_keywords_tf_idf))
#10
evaluations.append(get_evaluation(manual_keywords, mystem_predicted_keywords_graph_degree_centrality))
#11
evaluations.append(get_evaluation(manual_keywords, predicted_keywords_bigrams_tf_idf))

Отсортируем оценки по некоторым аспектам, для этого напишем функцию выдачи рейтинга по номеру аспекта

In [102]:
def get_aspect_rating(evaluations, aspect_number):
    """
    берет список оценок в формате dict.items() 
    и номер аспекта, где
    0 -- точность, 1 -- полнота,
    2 -- F-мера, 3 -- коэффициент Жаккара,
    и врзвращает рейтинг способов по убыванию оценки этого аспекта
    """
    rating = sorted(evaluations, key=lambda item: item[1][aspect_number], reverse=True)
    best = rating[0][0]
    worst = rating[-1][0]
    rating_to_print = []
    for index, item in enumerate(rating):
        mean_precision, mean_recall, fmeasure, jaccard_index = item[1]
        rating_to_print.append(f'{index + 1}. {item[0]}'
                               f'\nТочность: {mean_precision}\nПолнота: {mean_recall}\n'
                               f'F-мера: {fmeasure}\nКоэффициент Жаккара: {jaccard_index}\n')
    rating_to_print = '\n'.join(rating_to_print)
    return rating_to_print, best, worst

In [103]:
rating_to_print, best, worst = get_aspect_rating(evaluations, 2)

Рейтинг способов по F-мере:

In [104]:
print(rating_to_print)

1. pymorphy2_predicted_keywords_tf_idf
Точность: 0.12
Полнота: 0.27
F-мера: 0.16
Коэффициент Жаккара: 0.09

2. mystem_predicted_keywords_tf_idf
Точность: 0.12
Полнота: 0.27
F-мера: 0.16
Коэффициент Жаккара: 0.09

3. pymorphy2_predicted_keywords_graph_degree_centrality
Точность: 0.11
Полнота: 0.26
F-мера: 0.15
Коэффициент Жаккара: 0.09

4. mystem_predicted_keywords_most_frequent
Точность: 0.11
Полнота: 0.27
F-мера: 0.15
Коэффициент Жаккара: 0.09

5. mystem_predicted_keywords_graph_degree_centrality
Точность: 0.11
Полнота: 0.26
F-мера: 0.15
Коэффициент Жаккара: 0.09

6. predicted_keywords_bigrams_tf_idf
Точность: 0.11
Полнота: 0.26
F-мера: 0.15
Коэффициент Жаккара: 0.09

7. pymorphy2_predicted_keywords_most_frequent
Точность: 0.11
Полнота: 0.25
F-мера: 0.14
Коэффициент Жаккара: 0.08

8. pymorphy2_predicted_keywords_bigrams_most_frequent_without_verbs
Точность: 0.1
Полнота: 0.24
F-мера: 0.14
Коэффициент Жаккара: 0.08

9. mystem_predicted_keywords_bigrams_most_frequent
Точность: 0.1
Полнот

Для интереса можно посмотреть на первые несколько эталонных, лучших предсказанных и худших предсказанных групп ключевых слов

In [105]:
def print_manual_best_worst_keywords(best, worst, num_keywords_groups):
    """
    берет списки лучших и худших ключевых слов и количество групп ключевых слов,
    печатает их вместе с эталонными ключевыми словами
    """
    for one_text_manual, one_text_best, one_text_worst in zip(manual_keywords[:num_keywords_groups], 
                                                              globals()[best][:num_keywords_groups], 
                                                              globals()[worst][:num_keywords_groups]):
        print(f'Эталонные слова: {one_text_manual}'
              f'\nЛучший метод {best}: {one_text_best}'
              f'\nХудший метод {worst}: {one_text_worst}\n')

In [106]:
print_manual_best_worst_keywords(best, worst, 3)

Эталонные слова: ['школа', 'образовательные стандарты', 'литература', 'история', 'фгос']
Лучший метод pymorphy2_predicted_keywords_tf_idf: ['фгос', 'стандарт', 'васильев', 'образовательный', 'ольга', 'источник', 'предмет', 'предметный', 'исторический', 'произведение', 'письмо', 'редакция']
Худший метод pymorphy2_predicted_keywords_bigrams_most_frequent: ['ольга васильев', 'исторический источник', 'фгос', 'стандарт', 'источник', 'новый фгос', 'предметный результат', 'событие явления', 'образовательный', 'новый', 'ольга', 'васильев']

Эталонные слова: ['красота', 'законы']
Лучший метод pymorphy2_predicted_keywords_tf_idf: ['красота', 'отчаяние', 'порыв', 'кошка', 'глаз', 'руина', 'уйти', 'саби', 'ваби', 'банкнот', 'предмет', 'во']
Худший метод pymorphy2_predicted_keywords_bigrams_most_frequent: ['красота', 'глаз', 'отчаяние', 'уйти', 'порыв', 'глаз смотреть', 'уйти красотой', 'порыв отчаяния', 'красота уйти', 'кошка', 'красота кошки', 'оставаться мечтой']

Эталонные слова: ['юзефович', '